In [ ]:
import torch
import torch.nn as nn
from transformers import T5ForSequenceClassification, AutoTokenizer
import numpy as np

In [ ]:
from transformers import AutoModelForSequenceClassification

# Load mô hình
model_path = os.path.abspath("../VietnameseToxicityDectection")
tokenizer = AutoTokenizer.from_pretrained(model_path)
base_model = AutoModelForSequenceClassification.from_pretrained(model_path, local_files_only=True)
base_model.eval()

# Gói lại model để chỉ trả về logits
class WrappedModel(nn.Module):
    def __init__(self, model):
        super(WrappedModel, self).__init__()
        self.model = model

    def forward(self, input_ids, attention_mask=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.logits  # Chỉ lấy logits để dễ trace/script

# Gói model
wrapped_model = WrappedModel(base_model)
wrapped_model.eval()

# Tạo input mẫu
text = "Đây là một ví dụ."
inputs = tokenizer(text, return_tensors="pt")

# Trace
traced_model = torch.jit.trace(wrapped_model, (inputs["input_ids"], inputs["attention_mask"]))
traced_model.save("model.pt")
print("✅ TorchScript model saved as model.pt")


✅ TorchScript model saved as model.pt


In [ ]:
# Load tokenizer
model_path = os.path.abspath("../VietnameseToxicityDectection")
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load TorchScript model
model = torch.jit.load("model.pt")
model.eval()

# Test data
test_texts = [
    "Chúc bạn một ngày tốt lành!",
    "Đồ ngu dốt, không biết gì cả.",
    "Buổi thuyết trình hôm nay thật tuyệt vời.",
    "Mày là đồ vô học, cút đi!",
    "Cái nhà này rất tệ"
]

# Xử lý input và chạy mô hình
for text in test_texts:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        logits = model(inputs["input_ids"], inputs["attention_mask"])
        probs = torch.softmax(logits, dim=-1)  # softmax vì output có 2 lớp
        pred = torch.argmax(probs, dim=-1).item()  # lấy index lớp có xác suất cao nhất
        label = "TOXIC" if pred == 1 else "NON-TOXIC"
        print(f"📝 Text: {text}\n➡️ Prediction: {label} (score={probs[0, pred]:.4f})\n")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


📝 Text: Chúc bạn một ngày tốt lành!
➡️ Prediction: NON-TOXIC (score=0.9999)

📝 Text: Đồ ngu dốt, không biết gì cả.
➡️ Prediction: TOXIC (score=0.9957)

📝 Text: Buổi thuyết trình hôm nay thật tuyệt vời.
➡️ Prediction: NON-TOXIC (score=0.9984)

📝 Text: Mày là đồ vô học, cút đi!
➡️ Prediction: TOXIC (score=0.9761)

📝 Text: Cái nhà này rất tệ
➡️ Prediction: TOXIC (score=0.6490)



In [13]:
import torch
model = torch.jit.load("model.pt")

# In shape embedding
print(model.model.transformer.shared.weight.shape)  # ví dụ: torch.Size([vocab_size, embedding_dim])

# Nếu muốn lấy mảng numpy
embedding_weights = model.model.transformer.shared.weight.detach().cpu().numpy()
print(embedding_weights.shape)


torch.Size([36096, 768])
(36096, 768)


In [ ]:
# Tạo input mẫu giống với input cho model
text = "Đây là một ví dụ."
inputs = tokenizer(text, return_tensors="pt")

torch.onnx.export(
    wrapped_model,
    (inputs["input_ids"], inputs["attention_mask"]),
    "model.onnx",
    input_names=["input_ids", "attention_mask"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "seq_len"},
        "attention_mask": {0: "batch_size", 1: "seq_len"},
        "logits": {0: "batch_size"}
    },
    opset_version=14  # Đổi sang 14 để hỗ trợ triu và nhiều operator mới hơn
)
print("✅ ONNX model exported as model.onnx")


✅ ONNX model exported as model.onnx


In [ ]:
import os

# Load tokenizer
model_path = os.path.abspath("../VietnameseToxicityDectection")
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load ONNX model
onnx_model_path = "model.onnx"
ort_session = ort.InferenceSession(onnx_model_path)

# Test data
test_texts = [
    "Chúc bạn một ngày tốt lành!",
    "Đồ ngu dốt, không biết gì cả.",
    "Buổi thuyết trình hôm nay thật tuyệt vời.",
    "Mày là đồ vô học, cút đi!",
    "Cái nhà này rất tệ"
]

for text in test_texts:
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="np", padding=True, truncation=True)

    # Ép kiểu sang int64 vì ONNX Runtime yêu cầu
    input_feed = {
        "input_ids": inputs["input_ids"].astype(np.int64),
        "attention_mask": inputs["attention_mask"].astype(np.int64)
    }

    # Run inference
    logits = ort_session.run(output_names=["logits"], input_feed=input_feed)[0]  # [batch_size, num_labels]

    # Tính softmax để ra xác suất
    probs = np.exp(logits) / np.exp(logits).sum(axis=-1, keepdims=True)

    pred = np.argmax(probs, axis=-1)[0]
    label = "TOXIC" if pred == 1 else "NON-TOXIC"
    print(f"📝 Text: {text}\n➡️ Prediction: {label} (score={probs[0, pred]:.4f})\n")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


📝 Text: Chúc bạn một ngày tốt lành!
➡️ Prediction: NON-TOXIC (score=0.9999)

📝 Text: Đồ ngu dốt, không biết gì cả.
➡️ Prediction: TOXIC (score=0.9957)

📝 Text: Buổi thuyết trình hôm nay thật tuyệt vời.
➡️ Prediction: NON-TOXIC (score=0.9984)

📝 Text: Mày là đồ vô học, cút đi!
➡️ Prediction: TOXIC (score=0.9761)

📝 Text: Cái nhà này rất tệ
➡️ Prediction: TOXIC (score=0.6490)

